Tee yleiset muuttujanimet

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pyproj import CRS
from shapely import geometry, ops
from scipy.sparse.csgraph import connected_components

In [2]:
network2 = gpd.read_file("C:/data/L3Pilot/Katuverkko/Helsinki/hki_link_liikennemääräkartta2019/hki_liikennemaarat_2019_linkkikartta_3067_geomfix.shp") 
#network2 = gpd.read_file("C:/data/L3Pilot/Katuverkko/Helsinki/hki_link_liikennemääräkartta2019/hki_liikennemaarat_2019_linkkikartta_3067_geomfix_vclean.shp")
#network2 = gpd.read_file("C:/data/L3Pilot/Katuverkko/Helsinki/hki_link_liikennemääräkartta2019/test/Kehä1.shp")

In [3]:
collist = list(network2.columns)

In [4]:
network = gpd.GeoDataFrame(network2, columns=collist)

In [6]:
network["KATULUOKKA"] = network["KL"]

In [7]:
grouped = network.groupby("KATU")

In [8]:
#alternative katulist
katulist = list(network2["KATU"])
my_set = set(katulist)
uniquekadut = list(my_set)

In [9]:
len(uniquekadut)

463

In [10]:
#%%capture

testilista = []
mergelist = []
touchingbufslist2 = []
bufdflist2 = []
homogeeniset_linkit = gpd.GeoDataFrame()
mergetest = gpd.GeoDataFrame()

#for i in katulist: #Iterate over streets
for i in uniquekadut:
    grp = grouped.get_group(i)
    KLgrouped = grp.groupby("KL") #Group by road class
    
    #Iterate over classes in street
    for key, y in KLgrouped:
        KLgrp = KLgrouped.get_group(key)
        #Dissolve road sections that are of the same class in the same street
        dissolved = KLgrp.dissolve(by="KL") 
        dissolved = dissolved.reset_index()
        
        #Select sections which are not multilinestring. They can be added to main GDF.
        if str(type(dissolved.iloc[0, 1])) != "<class 'shapely.geometry.multilinestring.MultiLineString'>": 
            homogeeniset_linkit = homogeeniset_linkit.append(dissolved.iloc[0])
        
        #Select sections constisting of multilinestrings. These sections are not next to each other, and must be made separate for the main GDF
        elif str(type(dissolved.iloc[0, 1])) == "<class 'shapely.geometry.multilinestring.MultiLineString'>": 
            merge = ops.linemerge(dissolved.iloc[0, 1]) #now have the n. of continuous pieces as multilinestring object. Must be merged to join otherwise separate but connected lines.
            mergelist.append(merge)
            
       
                #BUFFERS --------- Buffers must be created around the seperate sections selected above. 
                #This is due to links in Helsinki network not always being spatially connected in the shape file (difference of 0.1-2m usually, 
                #even though they represent roads connected in reality. Buffers are used to bridge these links.            
           
        #Select sections merged to multilinestrings
            if str(type(merge)) == "<class 'shapely.geometry.multilinestring.MultiLineString'>": 
                dissolved3 = gpd.GeoDataFrame() #Create GDF to which we will collect these as seperate sections
                bufdf = gpd.GeoDataFrame() #Create GDF to which we will collect buffers of the seperate sections
                buflist = []
                for b in range(0, (len(merge))): #Create enough rows to accommodate all sections
                    dissolved3 = dissolved3.append(dissolved.iloc[0])
                
                #Put merge geometry in dissolved3
                for x in range(0, len(dissolved3)): 
                    dissolved3["geometry"].iloc[x] = merge[x]
                    
                dissolved3 = dissolved3.reset_index()
                testilista.append(dissolved3)
                
                #Create buffers for all seperated links
                for z in range(0, len(dissolved3)): 
                    buf = dissolved3["geometry"].iloc[z].buffer(8) #<---------ADJUST BUFFER SIZE HERE-------------
                    buflist.append(buf) #Add buffers to a list
                
                dissolved3 = dissolved3.drop(["index"], axis="columns") #Remove index column from dissolved3
                bufdf["geometry"] = buflist #Add contents of buflist to bufdf 
                bufdf.reset_index()
                bufdfseries =  bufdf["geometry"]
                bufdflist2.append(bufdf)
                
                
                #Check which buffers overlap. These are assigned a group value and they represent continuous sections
                overlap_matrix = bufdfseries.apply(lambda x: bufdfseries.overlaps(x)).values.astype(int)
                connected_components(overlap_matrix)
                n, ids = connected_components(overlap_matrix)
                touchingbufs = gpd.GeoDataFrame({'geometry': bufdfseries, 'group': ids})
                touchingbufslist2.append(touchingbufs)
                                                             
                               
                # Join with previously merged sections
                bufjoin = gpd.sjoin(dissolved3, touchingbufs, how="left", op="within") #Spatially join links (lines) with buffers
                bufjoin = bufjoin.reset_index()
                bufjoinfinal = bufjoin.dissolve(by="group") #Dissolve by BufferID
                bufjoinfinal = bufjoinfinal.reset_index()
                

                for n in range(0, len(bufjoin)): #Add links that were not within buffers
                    if pd.isna(bufjoin["group"].loc[n]):
                        bufjoinfinal = bufjoinfinal.append(bufjoin.iloc[n])
                
                for q in range(0, len(bufjoinfinal)):
                    homogeeniset_linkit = homogeeniset_linkit.append(bufjoinfinal.iloc[q]) # append final links to main GDF

            else: # Any other sections need no attention and can be passed to the main GDF
                dissolved4 = gpd.GeoDataFrame()
                dissolved4 = dissolved4.append(dissolved.iloc[0])
                dissolved4["geometry"].iloc[0] = merge
                homogeeniset_linkit = homogeeniset_linkit.append(dissolved4.iloc[0])
        

#homogeeniset_linkit = homogeeniset_linkit.drop(["level_0"], axis="columns")
homogeeniset_linkit = homogeeniset_linkit.reset_index()
homogeeniset_linkit.crs = CRS.from_epsg(3067)

In [11]:
mergelistnr = []
for i in range(0, len(mergelist)):
    if str(type(mergelist[i])) == "<class 'shapely.geometry.multilinestring.MultiLineString'>": 
        mergelistnr.append(i)

In [12]:
#Drop unnecessary columns
homogeeniset_linkit = homogeeniset_linkit.drop(["HA", "PA", "KA", "RA", "LA", "MP", "RV", "AUTOT", "pituus", "ID_NEW", "index", "index_right", "SY KAVL+RV"], axis="columns")

Calculating weighted mean of KVL for the continuous links:

In [13]:
# Create Link identifier
homogeeniset_linkit["H_ID"] = list(range(0, len(homogeeniset_linkit)))

In [14]:
#Spatial join between the continuous links we just created and the original link network
join = gpd.sjoin(homogeeniset_linkit, network2, how="left", op="contains")

In [15]:
joingrouped = join.groupby(by="H_ID")

In [16]:
grplist = []
for i in range(0, len(joingrouped)):
    grp = joingrouped.get_group(i)
    grp = grp.reset_index()
    kvlXpituus = []
    for j in range(0, len(grp)):
        kvlXpituus.append(grp.loc[j]["SY KAVL+RV"]*grp.loc[j]["pituus"])
    grp["kvlXpituus"] = kvlXpituus
    weightsum = sum(grp["pituus"])
    kvlsum = sum(grp["kvlXpituus"])
    weightedaverage = kvlsum/weightsum
    wmeangrpcol = list((np.repeat(weightedaverage, len(grp))))
    grp["WMEAN_KVL"] = wmeangrpcol
    grplist.append(grp)
    #weightedaveragelist.append(wmeangrpcol)
    

In [17]:
grpwmean = pd.concat(grplist)
grpwmean = grpwmean.drop(["kvlXpituus", "KATU_right", "index", "pituus", "KL_left", "KL_right", "LVUOSI_left", "linkki_left", "index_right", "linkki_right", "LVUOSI_right"], axis="columns")

In [18]:
grpwmean.columns

Index(['level_0', 'KATU_left', 'KATULUOKKA', 'geometry', 'group', 'H_ID', 'HA',
       'PA', 'KA', 'RA', 'LA', 'MP', 'RV', 'AUTOT', 'SY KAVL+RV', 'ID_NEW',
       'WMEAN_KVL'],
      dtype='object')

In [19]:
#Calculates mean KVL for continuous links
finaldissolve1 = grpwmean.dissolve(by='H_ID')

In [20]:
finaldissolve1.crs = CRS.from_epsg(3067)

In [22]:
len(finaldissolve1)

579

In [23]:
finaldissolve1["pitklink_pituus"] = finaldissolve1.geometry.length

In [24]:
homogeeniset_linkit.crs = CRS.from_epsg(3067)

In [25]:
finaldissolve1.crs = CRS.from_epsg(3067)

In [26]:
finaldissolve1 = finaldissolve1.drop(["level_0"], axis="columns")

In [172]:
#new_names = {'KATULUOKKA': 'Type', 'KATU_left': 'Section', 'H_ID': 'ID', "pitklink_pituus": "Length"}

#finaldissolve1 = finaldissolve1.rename(columns=new_names)

In [28]:
finaldissolve1.columns

Index(['geometry', 'KATU_left', 'KATULUOKKA', 'group', 'HA', 'PA', 'KA', 'RA',
       'LA', 'MP', 'RV', 'AUTOT', 'SY KAVL+RV', 'ID_NEW', 'WMEAN_KVL',
       'pitklink_pituus'],
      dtype='object')

In [160]:
finaldissolve1.to_file("C:/data/L3Pilot/Katuverkko/Helsinki/hki_link_liikennemääräkartta2019/finalproduct_newgroups_orig_wmean.shp")

In [47]:
#nulldissolve.to_csv("C:/data/L3Pilot/Katuverkko/Helsinki/hki_link_liikennemääräkartta2019/nulldissolve.csv")

In [ ]:
#finaldissolve1.to_file("C:/data/L3Pilot/Katuverkko/Helsinki/hki_link_liikennemääräkartta2019/finaldissolve.shp")

In [ ]:
#nulldissolve.to_file("C:/data/L3Pilot/Katuverkko/Helsinki/hki_link_liikennemääräkartta2019/nulldissolve2.shp")

In [ ]:
#homogeeniset_linkit.to_file("C:/data/L3Pilot/Katuverkko/Helsinki/hki_link_liikennemääräkartta2019/homogeeniset_linkit_mergetest_connectionsfixed.shp")

In [ ]:
#homogeeniset_linkit.to_csv("C:/data/L3Pilot/Katuverkko/Helsinki/hki_link_liikennemääräkartta2019/homogeeniset_linkit_mergetest_connectionsfixed.csv")